### Notebook Set-Up

#### Import Required Modules

In [0]:
# %pip install sqlalchemy duckdb duckdb-engine cloud-sql-python-connector[pg8000]

In [0]:
import os
import csv
import json
import duckdb

from typing import List, Dict
from dotenv import load_dotenv
from datetime import datetime

import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window

#### Load Secrets as Environment Variables

In [0]:
load_dotenv("/dbfs/FileStore/env/.env")

Out[49]: True

#### Get or Create SparkSession

In [0]:
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.setLogLevel("error")
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 spark://driver-proxy:7077 
 AppName 
 Databricks Shell

#### Set Workflow Constants

In [0]:
RAW_PATH = "dbfs:/FileStore/data/raw"
CLN_PATH = "dbfs:/FileStore/data/clean"
JSON_PATH = "/dbfs/FileStore/data/json"

### Export the Data as CSV Files to Bulk Insert into the Database

#### Import Cleaned Static Data

In [0]:
movies = spark.read.parquet(os.path.join(CLN_PATH, "movies"))
movies.show(5)
movies.count()

+-------+--------------------+------------+-------+--------------------+--------------------+---------+---------+----------+------------+----------+--------------------+
|tmdb_id|               title|release_date|runtime|              genres|            overview|   budget|  revenue|popularity|vote_average|vote_count|            keywords|
+-------+--------------------+------------+-------+--------------------+--------------------+---------+---------+----------+------------+----------+--------------------+
|  10010|      Brother Bear 2|  2006-08-17|   73.0|[Adventure, Anima...|Kenai finds his c...|     null|     null| 10.861154|         6.3|       318|[grizzly bear, hu...|
|  10012|              Cursed|  2005-02-25|   97.0|    [Horror, Comedy]|A werewolf loose ...| 35000000| 19294901|  8.949722|         5.1|       168|[brother sister r...|
| 100402|Captain America: ...|  2014-03-20|  136.0|[Action, Adventur...|After the catacly...|170000000|714766572| 18.717704|         7.6|      5881|[w

In [0]:
ratings = spark.read.parquet(os.path.join(CLN_PATH, "ratings"))
ratings.show(5)
ratings.count()

+-------+-------+------+-------------------+
|user_id|tmdb_id|rating|          timestamp|
+-------+-------+------+-------------------+
|  42017|     14|   4.0|2015-09-10 22:15:39|
|  54926|    165|   2.5|2006-03-19 20:54:43|
|  47629|   1637|   4.0|1996-07-01 14:27:03|
|  68508|  13920|   5.0|2003-11-14 01:18:56|
|  59910|  24746|   4.0|2017-07-29 19:41:07|
+-------+-------+------+-------------------+
only showing top 5 rows

Out[53]: 26024289

In [0]:
model_frame = spark.read.parquet(os.path.join(CLN_PATH, "model_frame"))
model_frame.show(5)
model_frame.count()

+-------+-------+------+-------------------+---------+----------+
|user_id|tmdb_id|rating|          timestamp|cnt_users|cnt_movies|
+-------+-------+------+-------------------+---------+----------+
| 171679|   2255|   2.0|2013-04-23 14:39:05|    13641|       133|
| 217667|  10427|   2.0|1996-11-08 17:21:30|     3441|        58|
|  31793|    532|   3.0|1997-05-07 15:14:38|    12698|       107|
|  96489|    197|   3.0|2015-10-25 19:42:18|    61076|        84|
| 198045|    641|   2.5|2015-11-04 04:07:02|    19478|       689|
+-------+-------+------+-------------------+---------+----------+
only showing top 5 rows

Out[54]: 15372729

#### Select the Subset of Users/Movies/Ratings Making it Into the Model Frame

In [0]:
model_users = model_frame.select('user_id').distinct().withColumn('user_id', f.col('user_id').cast('STRING')).filter(f.col('user_id').isNotNull())
model_users.count()

Out[55]: 195899

In [0]:
model_movies = model_frame.select('tmdb_id').distinct().withColumn('tmdb_id', f.col('tmdb_id').cast('STRING')).filter(f.col('tmdb_id').isNotNull())
model_movies.count()

Out[56]: 6235

In [0]:
model_ratings = model_frame \
    .select('user_id', 'tmdb_id').distinct() \
    .withColumn('user_id', f.col('user_id').cast('STRING')) \
    .withColumn('tmdb_id', f.col('tmdb_id').cast('STRING')) \
    .filter(f.col('user_id').isNotNull()) \
    .filter(f.col('tmdb_id').isNotNull())

model_ratings.count() 

Out[57]: 15372666

#### Create Users/Movies/Ratings Datasets for Export

In [0]:
updated_at = datetime.now()
updated_at

Out[58]: datetime.datetime(2023, 10, 23, 18, 44, 55, 294176)

In [0]:
export_users = model_users \
    .withColumn('fname', f.lit("ANONYMOUS")) \
    .withColumn('lname', f.lit("ANONYMOUS")) \
    .withColumn('email', f.lit("ANONYMOUS@ANONYMOUS.COM")) \
    .withColumn('updated_at', f.lit(updated_at))

export_users.agg(f.countDistinct('user_id').alias('users'), f.count('*').alias('records')).show()
export_users.printSchema()
export_users.show(5)

+------+-------+
| users|records|
+------+-------+
|195899| 195899|
+------+-------+

root
 |-- user_id: string (nullable = true)
 |-- fname: string (nullable = false)
 |-- lname: string (nullable = false)
 |-- email: string (nullable = false)
 |-- updated_at: timestamp (nullable = false)

+-------+---------+---------+--------------------+--------------------+
|user_id|    fname|    lname|               email|          updated_at|
+-------+---------+---------+--------------------+--------------------+
| 231350|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-23 18:44:...|
| 228408|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-23 18:44:...|
| 209326|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-23 18:44:...|
| 207398|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-23 18:44:...|
|  83250|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-23 18:44:...|
+-------+---------+---------+--------------------+--------------------+
only showing top 5 rows



In [0]:
export_movies = movies \
    .join(model_movies, on='tmdb_id', how='inner') \
    .select("tmdb_id", "title", "release_date", "runtime", "genres", "keywords", "overview", "budget", "revenue", "popularity", "vote_average", "vote_count") \
    .withColumn('updated_at', f.lit(updated_at))

export_movies.agg(f.countDistinct('tmdb_id').alias('movies'), f.count('*').alias('records')).show()
export_movies.printSchema()   
export_movies.show(5)

+------+-------+
|movies|records|
+------+-------+
|  6235|   6235|
+------+-------+

root
 |-- tmdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- runtime: double (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overview: string (nullable = true)
 |-- budget: integer (nullable = true)
 |-- revenue: integer (nullable = true)
 |-- popularity: double (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- updated_at: timestamp (nullable = false)

+-------+--------------------+------------+-------+--------------------+--------------------+--------------------+---------+---------+----------+------------+----------+--------------------+
|tmdb_id|               title|release_date|runtime|              genres|            keywords| 

In [0]:
export_ratings = ratings \
    .join(model_ratings, on=["user_id", "tmdb_id"], how='inner') \
    .withColumn("rank", f.row_number().over(Window.partitionBy("user_id", "tmdb_id").orderBy(f.rand(seed=1492)))) \
    .filter(f.col("rank") == 1) \
    .select("user_id", "tmdb_id", "rating") \
    .withColumn('updated_at', f.lit(updated_at))

export_ratings.agg(f.countDistinct('user_id', 'tmdb_id').alias('ratings'), f.count('*').alias('records')).show()
export_ratings.printSchema()   
export_ratings.show(5)

+--------+--------+
| ratings| records|
+--------+--------+
|15372666|15372666|
+--------+--------+

root
 |-- user_id: string (nullable = true)
 |-- tmdb_id: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- updated_at: timestamp (nullable = false)

+-------+-------+------+--------------------+
|user_id|tmdb_id|rating|          updated_at|
+-------+-------+------+--------------------+
| 100000|    280|   4.0|2023-10-23 18:44:...|
| 100001|  10158|   4.0|2023-10-23 18:44:...|
| 100002|  10923|   3.5|2023-10-23 18:44:...|
| 100002|  11529|   4.0|2023-10-23 18:44:...|
| 100002|    120|   4.0|2023-10-23 18:44:...|
+-------+-------+------+--------------------+
only showing top 5 rows



#### Export the Static User/Movie/Rating Data to JSONLINES

In [0]:
export_users.toPandas().to_json(os.path.join(JSON_PATH, "users.json"), orient="records", lines=True, date_format="iso")

In [0]:
export_movies.toPandas().to_json(os.path.join(JSON_PATH, "movies.json"), orient="records", lines=True, date_format="iso")

In [0]:
export_ratings.toPandas().to_json(os.path.join(JSON_PATH, "ratings.json"), orient="records", lines=True, date_format="iso")

### Load the JSON Exports into the Application Database Using Pandas

#### Import Relevant First-Party Modules

In [0]:
cnx = duckdb.connect('../app/database.duckdb')
cnx

### Populate the Database with the Static Kaggle Data

#### Populate the `movies` Table

In [0]:
result = cnx.execute(f"PRAGMA table_info('movies');").fetchall()
columns = [row[1] for row in result]
columns

['tmdb_id',
 'title',
 'release_date',
 'runtime',
 'genres',
 'keywords',
 'overview',
 'budget',
 'revenue',
 'popularity',
 'vote_average',
 'vote_count',
 'updated_at']

In [0]:
movies_spark = spark.read.parquet("../data/clean/movies").withColumn("updated_at", f.lit(datetime.now())).select(columns)
movies_spark.printSchema()
movies_spark.show(1, vertical=True)

root
 |-- tmdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- runtime: double (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overview: string (nullable = true)
 |-- budget: integer (nullable = true)
 |-- revenue: integer (nullable = true)
 |-- popularity: double (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- updated_at: timestamp (nullable = false)



-RECORD 0----------------------------
 tmdb_id      | 10010                
 title        | Brother Bear 2       
 release_date | 2006-08-17           
 runtime      | 73.0                 
 genres       | [Adventure, Anima... 
 keywords     | [grizzly bear, hu... 
 overview     | Kenai finds his c... 
 budget       | null                 
 revenue      | null                 
 popularity   | 10.861154            
 vote_average | 6.3                  
 vote_count   | 318                  
 updated_at   | 2023-10-21 21:33:... 
only showing top 1 row



In [0]:
movies_pandas = movies_spark.toPandas()
movies_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6918 entries, 0 to 6917
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   tmdb_id       6918 non-null   object        
 1   title         6918 non-null   object        
 2   release_date  6918 non-null   object        
 3   runtime       6918 non-null   float64       
 4   genres        6918 non-null   object        
 5   keywords      6674 non-null   object        
 6   overview      6918 non-null   object        
 7   budget        4252 non-null   float64       
 8   revenue       4252 non-null   float64       
 9   popularity    6918 non-null   float64       
 10  vote_average  6914 non-null   float64       
 11  vote_count    6916 non-null   float64       
 12  updated_at    6918 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(6), object(6)
memory usage: 702.7+ KB


In [0]:
cnx.execute("DELETE FROM movies")
cnx.register("movies_pandas", movies_pandas)
cnx.execute("INSERT INTO movies SELECT * FROM movies_pandas")
cnx.execute("CHECKPOINT")

#### Populate the `ratings` Table

In [0]:
result = cnx.execute(f"PRAGMA table_info('ratings');").fetchall()
columns = [row[1] for row in result]
columns

['user_id', 'tmdb_id', 'rating', 'updated_at']

In [0]:
ratings_spark = spark \
  .read.parquet("../data/clean/ratings") \
  .withColumn("updated_at", f.lit(datetime.now())) \
  .withColumn("rank", f.row_number().over(Window.partitionBy("user_id", "tmdb_id").orderBy(f.rand(seed=1492)))) \
  .filter(f.col("tmdb_id").isNotNull()) \
  .filter(f.col("rank") == 1) \
  .select(columns)

ratings_spark.printSchema()
ratings_spark.show(10)

root
 |-- user_id: string (nullable = true)
 |-- tmdb_id: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- updated_at: timestamp (nullable = false)



+-------+-------+------+--------------------+
|user_id|tmdb_id|rating|          updated_at|
+-------+-------+------+--------------------+
|      1|   9685|   5.0|2023-10-21 21:33:...|
|    100|    765|   3.5|2023-10-21 21:33:...|
|  10000|  82690|   5.0|2023-10-21 21:33:...|
| 100000|    280|   4.0|2023-10-21 21:33:...|
| 100001|  10158|   4.0|2023-10-21 21:33:...|
| 100002|  10923|   3.5|2023-10-21 21:33:...|
| 100002|  11529|   4.0|2023-10-21 21:33:...|
| 100002|    120|   4.0|2023-10-21 21:33:...|
| 100002|    121|   4.0|2023-10-21 21:33:...|
| 100002|  15472|   5.0|2023-10-21 21:33:...|
+-------+-------+------+--------------------+
only showing top 10 rows



In [0]:
ratings_splits = ratings_spark.randomSplit([0.1] * 10)

In [0]:
cnx.execute("DELETE FROM ratings")
for i, split in enumerate(ratings_splits):
    print(f"split={i}")
    ratings_pandas = split.toPandas()
    cnx.register("ratings_pandas", ratings_pandas)
    cnx.execute("INSERT INTO ratings SELECT * FROM ratings_pandas")
cnx.execute("CHECKPOINT")

split=0


split=1


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

split=2


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

split=8


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

split=9


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

#### Populate the `users` Table

In [0]:
result = cnx.execute(f"PRAGMA table_info('users');").fetchall()
columns = [row[1] for row in result]
columns

['user_id', 'fname', 'lname', 'email', 'updated_at']

In [0]:
users_spark = ratings_spark \
    .select('user_id') \
    .distinct() \
    .withColumn("fname", f.lit("ANONYMOUS")) \
    .withColumn("lname", f.lit("ANONYMOUS")) \
    .withColumn("email", f.lit("ANONYMOUS@ANONYMOUS.COM")) \
    .withColumn("updated_at", f.lit(datetime.now())) \
    .select(columns)

users_spark.show(10)
users_spark.count()

+-------+---------+---------+--------------------+--------------------+
|user_id|    fname|    lname|               email|          updated_at|
+-------+---------+---------+--------------------+--------------------+
| 100010|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-21 21:50:...|
| 100140|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-21 21:50:...|
| 100227|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-21 21:50:...|
| 100263|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-21 21:50:...|
| 100320|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-21 21:50:...|
| 100553|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-21 21:50:...|
| 100704|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-21 21:50:...|
| 100735|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-21 21:50:...|
| 100768|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-21 21:50:...|
|  10096|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-21 21:50:...|
+-------+---------+---------+--------------------+--------------

270888

In [0]:
users_pandas = users_spark.toPandas()
users_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270888 entries, 0 to 270887
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     270888 non-null  object        
 1   fname       270888 non-null  object        
 2   lname       270888 non-null  object        
 3   email       270888 non-null  object        
 4   updated_at  270888 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 10.3+ MB


In [0]:
cnx.execute("DELETE FROM users")
cnx.register("users_pandas", users_pandas)
cnx.execute("INSERT INTO users SELECT * FROM users_pandas")
cnx.execute("CHECKPOINT")

#### Close the Connection to Flush Changes to the Database

In [0]:
cnx.close()

### Test Out Query Functionality Using SQLAlchemy

In [0]:
from sqlalchemy import create_engine, MetaData, Table, select, text

In [0]:
engine = create_engine(f"duckdb:///../app/database.duckdb")
with engine.connect() as cnx:
    result = cnx.execute(text("SELECT * FROM movies LIMIT 5"))
    for row in result:
        print(row)

('10010', 'Brother Bear 2', datetime.date(2006, 8, 17), 73.0, ['Adventure', 'Animation', 'Family'], ['grizzly bear', 'human animal relationship', 'forest', 'hibernation', 'moose'], "Kenai finds his childhood human friend Nita and the two embark on a journey to burn the amulet he gave to her before he was a bear, much to Koda's dismay.", None, None, 10.861154, 6.3, 318, datetime.datetime(2023, 10, 21, 21, 33, 48, 25511))
('10012', 'Cursed', datetime.date(2005, 2, 25), 97.0, ['Horror', 'Comedy'], ['brother sister relationship', 'bite', 'transformation', 'supernatural powers', 'werewolf'], 'A werewolf loose in Los Angeles changes the lives of three young adults, who, after being mauled by the beast, learn they must kill their attacker if they hope to change their fate to avoid becoming werewolves too.', 35000000, 19294901, 8.949722, 5.1, 168, datetime.datetime(2023, 10, 21, 21, 33, 48, 25511))
('100402', 'Captain America: The Winter Soldier', datetime.date(2014, 3, 20), 136.0, ['Action', 

In [0]:
cnx.close()